### Object detection and tracking with opencv

In [1]:
import cv2

In [2]:
def initialize(videopath):
    cap = cv2.VideoCapture(videopath)
    od = cv2.createBackgroundSubtractorMOG2(varThreshold=50, history=100) # better selection and quicker
    # od = cv2.createBackgroundSubtractorKNN()
    return cap, od

In [3]:
cap, od =initialize('../data/aerial_football_video.mp4')

In [4]:
def identifying_objects(c, od):
    count = 0
    while True:
        ret, frame = c.read()                                       # read frames
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        frame = cv2.resize(frame, (960, 540))                       # Resize image
        cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY) # set treshold to only get really contracting values in themask
        cv2.imshow('mask', mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        detect = []
        for cnt in contours:
            count += 1
            x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
            detect.append([x,y,w,h])
            cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
            cv2.imshow('frame2', frame)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)              
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return c, frame

In [5]:
cap, frame = identifying_objects(cap, od)

In [6]:
## first step was finding objects, now its time for tracking

In [6]:
tracker = cv2.legacy.TrackerMOSSE_create()

In [7]:
def trackng_objects(c, od):
    frame_count = 0
    while True:
        ret, frame = c.read()                                       # read frames
        frame_count += 1
        frame = cv2.resize(frame, (960, 540))                       # Resize image
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY) # set treshold to only get really contracting values in themask
        cv2.imshow('mask', mask)
        bbox = cv2.selectROI(frame, False)                        # handmatige select bbox op video
        if frame_count == 1:
            tracker.init(frame, bbox)
            print('initializing tracker')
            print(bbox, tracker)        
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        detect = []
        for cnt in contours:
            if frame_count > 1:
                x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
                ret, bbox = tracker.update(frame)                      
                print(bbox)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2, 1)

                # cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
                cv2.imshow('frame2', frame)
                frame_count += 1
        # box_ids = tracker.update(detect)
        # for box_id in box_ids:
        #     x,y,w,h, id = box_id
        #     cv2.putText(frame, str(id), (x, y-15), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),2)
        #     cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 3)
        #     print(box_ids)
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return c, frame

In [8]:
trackng_objects(cap, od)

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
initializing tracker
(383, 362, 26, 32) < cv2.legacy.TrackerMOSSE 0x7fc0a96ddb70>
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [ ]:
### to do:

## define in first frame the bbox of specific obecjt
## create tracker for that bbox
## update tracker per frame